In [71]:
import numpy as np
import matplotlib.pyplot as plt
from medpy import io
from radiomics import featureextractor, imageoperations
import six
import os
import pandas as pd
import SimpleITK as sitk
import seaborn as sns
from sklearn.metrics import roc_auc_score
import radiomics


import sys
sys.path.append('../../')


%load_ext autoreload
%autoreload 2
from radiomics_util import *

from sklearn.linear_model import Lasso, SGDClassifier, LogisticRegression
from sklearn.model_selection import cross_validate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
gt = pd.read_csv("/data1/practical-sose23/morphometric/picai_labels_cropped/clinical_information/marksheet_filled_simple_normalized.csv")
gt.head()

,Unnamed: 0,patient_id,study_id,patient_age,psa,psad,prostate_volume,case_ISUP,case_csPCa,original_shape_Elongation_T2W,...,original_gldm_LargeDependenceEmphasis_ADC,original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC,original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC,original_gldm_LowGrayLevelEmphasis_ADC,original_gldm_SmallDependenceEmphasis_ADC,original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC,original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC,ellipse_vol_ADC,patient_id_ADC,prostate_volume_ADC
0,0,10000,1000000,1.029209,-0.287234,2.461960e-16,-0.279400,-0.705929,0,-1.175607,...,-2.051181,0.430926,-0.544297,-0.506616,1.417104,0.643569,-0.449200,-0.699413,-1.728815,-0.279400
1,1,10001,1000001,-0.222329,-0.219422,-3.974937e-01,1.021754,-0.705929,0,-0.015216,...,1.174161,0.648656,-0.584759,-0.602858,-1.189592,0.488914,-0.665584,1.432062,-1.726463,1.021754
2,2,10002,1000002,-1.056687,-0.524577,-4.861952e-01,0.246598,-0.705929,0,1.374727,...,-0.589672,0.300434,-0.333223,-0.290034,0.601330,0.264669,-0.182080,0.244342,-1.724110,0.246598
3,3,10003,1000003,0.890150,0.072170,2.461960e-16,0.177388,-0.705929,0,1.338907,...,0.864472,0.634531,-0.587662,-0.601442,-0.720011,0.522172,-0.632072,0.257059,-1.721758,0.177388
4,4,10004,1000004,0.194851,-0.266891,-3.679266e-01,0.357335,-0.705929,0,-0.504788,...,0.448343,0.616729,-0.592742,-0.602729,-0.532201,0.560409,-0.643932,0.493917,-1.719406,0.357335


In [105]:
df = gt.drop(["case_csPCa", "patient_id", "study_id", "case_ISUP"], axis=1) #gt[["patient_age", "psa", "psad", "prostate_volume"]] 
print(df.columns)
target = gt[ "case_csPCa"]
# Initialize the Lasso model
lasso = LogisticRegression(max_iter=10000)  # You can adjust the value of alpha (regularization strength) as needed

result = cross_validate(lasso, X=df, y=target, cv=5, scoring="roc_auc", return_train_score=True, return_estimator=True)
print(result)
estimators = result["estimator"]
test_scores = result["test_score"]
train_scores = result["train_score"]
print("test scores: ", test_scores.mean())
print("train scores:", train_scores.mean())

#features = np.asarray([])
#for estimator in estimators:
#    feature_importances = estimator.coef_
#    # Create a mask to identify the selected features
#    selected_features = df.iloc[:,:-1].columns[feature_importances != 0]
#    features = np.concatenate((features, selected_features))
#    # Print the selected features
#    print("Selected features:", list(selected_features))

Index(['Unnamed: 0', 'patient_age', 'psa', 'psad', 'prostate_volume',
       'original_shape_Elongation_T2W', 'original_shape_Flatness_T2W',
       'original_shape_LeastAxisLength_T2W',
       'original_shape_MajorAxisLength_T2W',
       'original_shape_Maximum2DDiameterColumn_T2W',
       ...
       'original_gldm_LargeDependenceEmphasis_ADC',
       'original_gldm_LargeDependenceHighGrayLevelEmphasis_ADC',
       'original_gldm_LargeDependenceLowGrayLevelEmphasis_ADC',
       'original_gldm_LowGrayLevelEmphasis_ADC',
       'original_gldm_SmallDependenceEmphasis_ADC',
       'original_gldm_SmallDependenceHighGrayLevelEmphasis_ADC',
       'original_gldm_SmallDependenceLowGrayLevelEmphasis_ADC',
       'ellipse_vol_ADC', 'patient_id_ADC', 'prostate_volume_ADC'],
      dtype='object', length=215)
{'fit_time': array([0.86611795, 0.88964105, 0.86850238, 0.86715698, 0.77150536]), 'score_time': array([0.00381517, 0.00382042, 0.00524783, 0.00406456, 0.00505304]), 'estimator': [LogisticRegre